Заказчик проведения данной работы - отдел маркетинга.  

Информация нужна менеджеру, который занимается маркетинговыми компаниями и работает в тесной связке с менеджерами продуктов.   

На выходе хотят получить сегменты пользователей по продуктам, который им подходят, либо сигменты по количеству используемых продуктов и признакам отточности и неотточности. 

Банк хочет переработать продуктовую линейку, чтобы удержать отточных клиентов, а также привлечь лояльных банку клиентов.   

In [1]:
# Загружаем библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math as mth
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')
import scipy.stats as st
import datetime as dt
import squarify 
import plotly.express as px
import plotly.subplots as sp
from plotly import graph_objects as go
from collections import Counter
from pandas.plotting import register_matplotlib_converters

In [2]:
# Указываем базовые настройки 
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
register_matplotlib_converters()

In [3]:
# Указываем настройки графиков
large = 16; med = 10; small = 8
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (10, 6),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
sns.set_style("whitegrid")
%matplotlib inline

In [4]:
# Указываем путь к файлу
try:
    data_raw = pd.read_csv('C:/Users/Boris/Desktop/datasets/bank_scrooge.csv')
except: 
    data_raw = pd.read_csv('/datasets/bank_scrooge.csv')

In [5]:
# Выведем первые 5 строчек набора данных
data_raw.head()

,USERID,score,city,gender,age,equity,balance,products,credit_card,last_activity,EST_SALARY,churn
0,183012,850.00,Рыбинск,Ж,25.00,1,59214.82,2,0,1,75719.14,1
1,146556,861.00,Рыбинск,Ж,37.00,5,850594.33,3,1,0,86621.77,0
2,120722,892.00,Рыбинск,Ж,30.00,0,NaN,1,1,1,107683.34,0
3,225363,866.00,Ярославль,Ж,51.00,5,1524746.26,2,0,1,174423.53,1
4,157978,730.00,Ярославль,М,34.00,5,174.00,1,1,0,67353.16,1


In [6]:
# Посмотрим информацию по данным в таблице
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   USERID         10000 non-null  int64  
 1   score          10000 non-null  float64
 2   city           10000 non-null  object 
 3   gender         10000 non-null  object 
 4   age            9974 non-null   float64
 5   equity         10000 non-null  int64  
 6   balance        7705 non-null   float64
 7   products       10000 non-null  int64  
 8   credit_card    10000 non-null  int64  
 9   last_activity  10000 non-null  int64  
 10  EST_SALARY     10000 non-null  float64
 11  churn          10000 non-null  int64  
dtypes: float64(4), int64(6), object(2)
memory usage: 937.6+ KB


Описание данных:  
**`USERID`** — идентификатор пользователя,  
**`score`** — баллы кредитного скоринга,  
**`city`** — город,  
**`gender`** — пол,  
**`age`** — возраст,  
**`equity`** — количество баллов собственности,  
**`balance`** — баланс на счёте,  
**`products`** — количество продуктов, которыми пользуется клиент,  
**`credit_card`** — есть ли кредитная карта,  
**`last_activity`** — активный клиент,  
**`EST_SALARY`** — оценочный доход клиента,  
**`сhurn`** — признак оттока.  

Нам известно, что столбцы **score**, **equality** и **est_salary** являются внешними для банка. Т.е. получаются из внешних систем и являются композитной оценкой. 

In [7]:
data_raw.describe()

,USERID,score,age,equity,balance,products,credit_card,last_activity,EST_SALARY,churn
count,10000.00,10000.00,9974.00,10000.00,7705.00,10000.00,10000.00,10000.00,10000.00,10000.00
mean,171814.71,848.70,42.73,2.63,827794.31,1.87,0.68,0.52,147866.89,0.18
std,33708.24,65.45,12.18,1.98,1980614.15,0.79,0.47,0.50,139388.51,0.39
min,94561.00,642.00,18.00,0.00,0.00,0.00,0.00,0.00,2546.30,0.00
25%,142810.25,802.00,33.00,0.00,295554.16,1.00,0.00,0.00,75251.90,0.00
50%,172728.00,853.00,40.00,3.00,524272.20,2.00,1.00,1.00,119658.10,0.00
75%,201261.75,900.00,51.00,4.00,980705.85,2.00,1.00,1.00,174500.54,0.00
max,229145.00,1000.00,86.00,9.00,119113552.01,5.00,1.00,1.00,1395064.45,1.00


## Предварительная обработка данных

In [17]:
# Переименуем столбцы
data = data_raw.copy()
data = data.rename(columns={'USERID':'user_id'})
data.columns = data.columns.str.lower()

In [31]:
data['age'].isna().sum()

26

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        10000 non-null  int64  
 1   score          10000 non-null  float64
 2   city           10000 non-null  object 
 3   gender         10000 non-null  object 
 4   age            9974 non-null   float64
 5   equity         10000 non-null  int64  
 6   balance        7705 non-null   float64
 7   products       10000 non-null  int64  
 8   credit_card    10000 non-null  int64  
 9   last_activity  10000 non-null  int64  
 10  est_salary     10000 non-null  float64
 11  churn          10000 non-null  int64  
dtypes: float64(4), int64(6), object(2)
memory usage: 937.6+ KB


In [30]:
data.head(5)

,user_id,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
0,183012,850.00,Рыбинск,Ж,25.00,1,59214.82,2,0,1,75719.14,1
1,146556,861.00,Рыбинск,Ж,37.00,5,850594.33,3,1,0,86621.77,0
2,120722,892.00,Рыбинск,Ж,30.00,0,NaN,1,1,1,107683.34,0
3,225363,866.00,Ярославль,Ж,51.00,5,1524746.26,2,0,1,174423.53,1
4,157978,730.00,Ярославль,М,34.00,5,174.00,1,1,0,67353.16,1
